In [213]:
import sounddevice as sd
import random
import numpy as np
import librosa

from pydub import AudioSegment
from scipy.io import wavfile
from midi2audio import FluidSynth
from midiutil import MIDIFile
from src.util.features import get_feature_vector
from scipy.spatial.distance import cosine, minkowski

from mido import MidiFile, Message, MidiTrack

In [197]:
def play(file=None, sr=44100):
    if file == None: file=WAV_FILE+".wav"
    song, fs = librosa.load(file,sr=sr)
    sd.play(song, fs)
    sd.wait()

In [4]:
def rand(low=0, high=2):
    return random.random()*(high - low + 1) + low

In [351]:
FONT = "./src/soundfonts/piano_chords.sf2"
MIDI_FILE = "./src/input/temp"
WAV_FILE = "./src/output/temp"

In [305]:
def save_harmony(midi_file, out):
    with open(MIDI_FILE + out + ".midi", "wb") as output_file:
        midi_file.writeFile(output_file)
        output_file.close()
        
    song = BytesIO()
    wav_song = BytesIO()
    
    midi_file.writeFile(song)

    FluidSynth(
            sound_font=FONT
        ).midi_to_audio(MIDI_FILE + out + ".midi", WAV_FILE + out + ".wav")

In [315]:
"""

Feito pelo GPT

"""
def generate_random_chord_progression():
    chords = [[60, 64, 67], [62, 65, 69], [64, 67, 71], [65, 69, 72]]
    return random.choices(chords, k=4)

def generate_random_melody():
    # Create a new MIDI file
    mf = MIDIFile(2)
    melody_track = 0
    bassline_track = 1
    time = 0
    
    mf.addTrackName(melody_track, time, "Melody Track")
    mf.addTrackName(bassline_track, time, "Bassline Track")
    mf.addTempo(melody_track, time, 120)
    mf.addTempo(bassline_track, time, 120)

    # Generate a random chord progression
    chord_progression = generate_random_chord_progression()

    for i in range(10):
        # Add the chord to the MIDI file
        chord = chord_progression[i % len(chord_progression)]
        for n in chord:
            mf.addNote(melody_track, 0, n, i, 1, 100)

        # Generate a random melody note that fits the chord
        pitch = random.choice(chord)
        duration = 1
        mf.addNote(melody_track, 0, pitch, i, duration, 100)

        # Generate a random bassline note that fits the chord
        print(chord)
        current_chord = [note for note in chord if note % 12 in [0, 5]]
        if len(current_chord)==0:
            current_chord = [chord[0]]
        bassline_pitch = random.choice(current_chord)
        mf.addNote(bassline_track, 0, bassline_pitch, i, 1, 100)

    # Save the MIDI file
    save_harmony(mf, "")

In [267]:
generate_random_melody()

[60, 64, 67]
[60, 64, 67]
[65, 69, 72]
[64, 67, 71]
[60, 64, 67]
[60, 64, 67]
[65, 69, 72]
[64, 67, 71]
[60, 64, 67]
[60, 64, 67]


## Parameters to encode
 - [ ] Tempo
 - [ ] Nº of Notes
 - [x] Notes
 - [x] Notes duration
 - [x] Chords
 - [ ] Probability of operations (?)

In [308]:
def get_pitch(i: int):
    pitch_base = [23, 35, 47, 59, 71, 83, 95, 107]
    return pitch_base[i]

In [ ]:
def generate_random_note():
    pitch = get_pitch(3)
    notes = list(range(0,12+1))
    
    note = random.choice(notes) + pitch
    return note

In [443]:
def generate_random_chord():
    chord = [generate_random_note() for _ in range(3)]
    duration = random.choice(range(1, 8+1))
    return duration, chord

In [364]:
generate_random_meldody()

In [437]:
def generate_random_genome():
    N = random.choice(range(5, 20+1))
    
    genome = []
    for _ in range(N):
        duration, chord = generate_random_chord()
        melody_note = random.choice(chord)
        genome.append((duration/2, chord, melody_note))
    # ( Duration, Chord, Note )
    return genome

In [449]:
def get_phenotype(genome):
    midi_file = MIDIFile(2)
    melody=0
    
    tempo = 120
    midi_file.addTempo(melody, 0, tempo)
    
    cum_time = 0
    for el in genome:
        d, chord, note = el
        if 59 not in chord:
            for n in chord:
                midi_file.addNote(melody, 0, n, cum_time, d, 100)
            midi_file.addNote(melody, 0, note, cum_time, d, 100)
        
        cum_time+=d
    save_harmony(midi_file, "")
    return midi_file

In [447]:
gen = generate_random_genome()
gen

[(3.0, [71, 69, 64], 64),
 (1.5, [70, 62, 64], 62),
 (1.0, [70, 70, 64], 70),
 (3.0, [65, 67, 71], 67),
 (1.5, [67, 66, 70], 66),
 (1.0, [59, 63, 59], 63),
 (1.5, [66, 61, 71], 71)]

In [450]:
mf = get_phenotype(gen)